In [1]:
using Oscar

In [ ]:
function dimQI(v,lista)
    # devuelve el grado de la variable v en el anillo cociente definido por una lista de polinomios
    max = 0
    leadmons = Set([leading_monomial(f) for f in lista]) # crea un conjunto de los monomios principales de cada polinomio
    for p in leadmons
        if v in vars(p) # verifica si la variable esta en las variables de el monomio principal
            m = degree(p, v) # calcula el grado de v en el monomio
            if m> max # busca el mayor valor
                max = m
            end
        end
    end
    return max # devuelve el maximo grado de v en el anillo cociente determinado por la lista de polinomios
end

In [ ]:
function kbasis(GB, VList)
    # devuelve una lista de monomios que forman una base del anillo cociente como espacio vectorial, donde GB es una base de Gröbner para un ideal de dimensión cero, 
    #y genera un mensaje de error si el ideal no es de dimensión cero.
    
    if iszero(dim(GB))  # verifica si el ideal es de dimensión cero
        Gb = collect(gens(GB))
        B = [(div(VList[1],VList[1])) ]  # inicializa la lista de monomios de la base con el monomio 1
        leadmons = Set([leading_monomial(f) for f in Gb])  # crea un conjunto de los monomios principales de cada polinomio en la base de Gröbner
        for v in VList
            m = dimQI(v, leadmons)  # determina el grado de la variable v en el anillo cociente
            C = copy(B)  # crea una copia de la lista de monomios actual como base para la siguiente iteración
            for t in C
                for l in 1:m-1
                    t = t*v  # multiplica el monomio actual por la variable v
                    if  !(any((divrem(t,u)[2] == 0) for u in leadmons))  # verifica si alguno de los monomios principales de la base de Gröbner divide al nuevo monomio
                        push!(B, t)  # si ningún monomio principal lo divide, agrega el nuevo monomio a la lista de monomios de la base
                    end
                end
            end
        end
        return B  # devuelve la lista de monomios de la base
    else
        error("el ideal no es de dimensión cero")
    end
end

In [2]:
function matrixM(variable,kbase,Gbase)
    # Dada una variable, una kbase de monomios del anillo cociente y una base de Groebner. Calcula la matriz de multiplicacion por la variable 
    kbx = kbase*variable 
    reducida = reduce(collect(gens(kbx)),collect(Gbase)) # Encuentra la representacion en el anillo cociente de los elementos de  labase por la variable
    Mx = [coeff(kr,k) for k in collect(gens(kbase)) , kr in collect(reducida)] # Crea a la matriz
    return Mx
end

matrixM (generic function with 1 method)

In [ ]:
GR, x = PolynomialRing(QQ, ["x$i" for i in 1:nv(g)]) # Definimos el anillo de polinomios con n = #vercies del grafo variables
function polisGrafo(g,k,GR)
    # Devuelve dado un grafo y un entero k, un sistema de polinomios que ayuda a determinar que el grafo es k-colorable Y el anillo de polinomios
    ps = [GR[j]^k-1 for j in 1:nv(g)] # los polinomios x^k_i- 1 = 0, ∀i ∈ V(g)
    ps2 = [GR[src(r)]^(k-1-l) * GR[dst(r)]^(l) for r in collect(edges(g)), l in 0:k-1] # los polinomios sum_l=0^{k-1} x_i^{k-1-l} x_j^l=0 ∀{i, j} ∈ E(G)
    p2 = sum(ps2,dims=2)
    polis = vec(vcat(ps,p2)) # crea una lista con los polinomios
    return polis # retorna la lista de polinomios 
end